# DCA Analysis

Scenarios:
1. DCA BTC
    * Every 14 days buy all in BTC<br>
2. DCA ETH
    * Every 14 days buy all in ETH<br>
3. DCA BTC & ETH
    * Buy BTC and ETH every 14 days at market price (50/50 split)<br>
4. DCA BTC & ETH (Re-balance every month)
    * Buy BTC and ETH every 14 days but re-balance so full portfolio is 50/50 split on first of month<br>
5. DCA Top 4 (BTC, ETH, EOS, LTC)
    * Buy top 4 crypto at market price every 14 days (25% splits)<br>
6. DCA Top 4 (BTC, ETH, EOS, LTC)
    * Buy top 4 crypto every 14 days but re-balance full portfolio every 1st of month<br>

## Setup

In [55]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import yfinance as yf
from yahoofinancials import YahooFinancials
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
#Initial Variables

In [57]:
start_date = '2019-11-01'
end_date = '2020-11-01'

#amount of BTC mined per day = 0.0003 (conservative)
mine_day_amt = 0.0003

#purchasing days = 1st and 14th
purchase_days = [1, 15]




In [154]:
#get prices for period
crypto_df = yf.download('BTC-USD ETH-BTC EOS-BTC LTC-BTC ETH-USD', 
                      start=start_date, 
                      end=end_date,
                        auto_adjust=True)

#clean data
crypto_df = crypto_df.drop(columns=['Open', 'Volume', 'High', 'Low'])
crypto_df.columns = crypto_df.columns.droplevel(0)
#crypto_df.drop('Close', axis=1, level=1)
#crypto_df.index = pd.to_datetime(crypto_df.index)

#adjust dates
#reset_start = datetime.strptime(start_date, '%Y-%m-%d')
#reset_end = datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=1)
#total_days = reset_end - reset_start

crypto_df

[*********************100%***********************]  5 of 5 completed


,BTC-USD,EOS-BTC,ETH-BTC,ETH-USD,LTC-BTC
Date,,,,,
2019-10-31,9199.584961,0.000356,0.019997,183.966919,0.006389
2019-11-01,9261.104492,0.000362,0.019865,183.969894,0.006326
2019-11-02,9324.717773,0.000358,0.019725,183.925720,0.006284
2019-11-03,9235.354492,0.000356,0.019753,182.425018,0.006348
2019-11-04,9412.612305,0.000366,0.019798,186.355194,0.006542
...,...,...,...,...,...
2020-10-27,13654.218750,0.000195,0.029588,403.997040,0.004234
2020-10-28,13271.285156,0.000200,0.029285,388.650757,0.004200
2020-10-29,13437.882812,0.000196,0.028779,386.730103,0.004075


In [168]:
crypto_df.loc['2019-10-31':'2019-11-15']

,BTC-USD,EOS-BTC,ETH-BTC,ETH-USD,LTC-BTC
Date,,,,,
2019-10-31,9199.584961,0.000356,0.019997,183.966919,0.006389
2019-11-01,9261.104492,0.000362,0.019865,183.969894,0.006326
2019-11-02,9324.717773,0.000358,0.019725,183.925720,0.006284
2019-11-03,9235.354492,0.000356,0.019753,182.425018,0.006348
2019-11-04,9412.612305,0.000366,0.019798,186.355194,0.006542
2019-11-05,9342.527344,0.000387,0.020263,189.304169,0.006776
2019-11-06,9360.879883,0.000387,0.020468,191.593842,0.006866
2019-11-07,9267.561523,0.000377,0.020283,187.976547,0.006673
2019-11-08,8804.880859,0.000389,0.020922,184.211472,0.006875


## Scenario 1: DCA BTC every 14 days

In [169]:
s1_total_btc = mine_day_amt * total_days.days

In [170]:
s1_total_btc

0.10949999999999999

In [301]:
s1_usd_val = s1_total_btc * crypto_df.tail(1).loc[:, 'BTC-USD']
s1_usd_val = pd.DataFrame(s1_usd_val)
s1_usd_val

,BTC-USD
Date,
2020-10-31,1509.018965


## Scenario 2: DCA ETH every 14 days

In [174]:
month_list = [i.strftime("%Y-%m-%d") for i in pd.date_range(start=reset_start, end=reset_end, freq='SMS')]

s2_prices_df = crypto_df.loc[month_list, :]
s2_prices_df = pd.DataFrame(s2_prices_df.loc[:, 'ETH-BTC'])
s2_purchases_df = (mine_day_amt * 14) / s2_prices_df 

In [175]:
s2_total_eth = s2_purchases_df.sum()
s2_total_eth

ETH-BTC    4.158532
dtype: float64

In [176]:
s2_usd_val = s2_total_eth * crypto_df.tail(1).loc[:, 'ETH-USD'].values[0]

#s2_total_eth * crypto_df.tail(1).loc[:, 'ETH-USD']
s2_usd_val
#s2_usd_val = s2_total_eth * crypto_df.tail(1).loc[:, 'ETH-USD']
#s2_usd_val

ETH-BTC    1607.648214
dtype: float64

## Scenario 3: DCA BTC and ETH every 14 days

In [254]:
s3_prices_df = crypto_df.loc[month_list, :]
s3_prices_df = pd.DataFrame(s3_prices_df.loc[:, ['ETH-BTC', 'BTC-USD', 'ETH-USD']])

#total purchase for ETH is half mining amount and sum
s3_purchases_df = s3_prices_df

s3_purchases_df

,ETH-BTC,BTC-USD,ETH-USD
Date,,,
2019-11-01,0.019865,9261.104492,183.969894
2019-11-15,0.021258,8491.992188,180.521179
2019-12-01,0.020364,7424.292480,151.185730
2019-12-15,0.020010,7152.301758,143.114990
2020-01-01,0.018167,7200.174316,130.802002
2020-01-15,0.018875,8807.010742,166.230682
2020-02-01,0.019555,9392.875000,183.673950
2020-02-15,0.026769,9889.424805,264.728577
2020-03-01,0.025573,8562.454102,218.970596


In [291]:

#s3_purchases_df.loc[:, 'BTC-USD']
s3_purchases_df['ETH Units'] = (mine_day_amt * 7) / s3_purchases_df['ETH-BTC']
s3_purchases_df['BTC Units'] = (mine_day_amt * 7)
s3_purchases_df['ETH in BTC'] = s3_purchases_df['ETH Units'] * s3_purchases_df['ETH-BTC']
s3_total_eth_units = s3_purchases_df['ETH Units'].sum()
s3_total_btc_units = s3_purchases_df['BTC Units'].sum()

#BTC Equivalent
s3_total_eth_inbtc = s3_total_eth_units * s3_purchases_df.tail(1)['ETH-BTC']
#USD Equivalent
s3_total_eth_inusd = s3_total_eth_units * s3_purchases_df.tail(1)['ETH-USD']
s3_total_btc_inusd = s3_total_btc_units * s3_purchases_df.tail(1)['BTC-USD']


s3_total_usd = pd.DataFrame(index=s3_purchases_df.tail(1).index)
s3_total_usd['Total ETH Units'] = s3_total_eth_units
s3_total_usd['Total BTC Units'] = s3_total_btc_units
s3_total_usd['Total ETH in BTC'] = s3_total_eth_inbtc
s3_total_usd['Total ETH in USD'] = s3_total_eth_inusd
s3_total_usd['Total BTC in USD'] = s3_total_btc_inusd
s3_total_usd['Total Port. BTC Value'] = s3_total_btc_units + s3_total_eth_inbtc
s3_total_usd['Total Port. USD Value'] = s3_total_eth_inusd + s3_total_btc_inusd

In [292]:
s3_total_usd

,Total ETH Units,Total BTC Units,Total ETH in BTC,Total ETH in USD,Total BTC in USD,Total Port. BTC Value,Total Port. USD Value
Date,,,,,,,
2020-10-15,2.079266,0.0504,0.068271,784.801946,579.36562,0.118671,1364.167566
